In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [73]:
data = loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [74]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [75]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [76]:
def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / (2 * len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))#正则化
    return np.sum(first - second) / len(X) + reg

In [77]:
#向量化梯度函数
def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

In [78]:
#一对多分类器
from scipy.optimize import minimize

def one_vs_all(X,y,num_labels,learning_rate):
    rows=X.shape[0]
    params=X.shape[1]

    all_theta=np.zeros((num_labels,params+1))#第i行代表数字i的分类器
    X=np.insert(X,0,values=np.ones(rows),axis=1)#axis:插入的维度。插入ones

    for i in range(1,num_labels+1):
        theta=np.zeros(params+1)#初始化猜测值
        y_i=np.array([1 if label==i else 0 for label in y])
        y_i=np.reshape(y_i,(rows,1))#对于i的一对一分类

        fmin=minimize(fun=cost,x0=theta,args=(X,y_i,learning_rate),method='TNC',jac=gradient)#最小化目标函数
        all_theta[i-1:]=fmin.x
    return all_theta

In [79]:
np.unique(data['y'])#看下有几类标签

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [80]:
all_theta=one_vs_all(data['X'],data['y'],10,1)
all_theta

array([[-2.38329625e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30469860e-03, -8.20996452e-10,  0.00000000e+00],
       [-3.18487445e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.46292788e-03, -5.08767264e-04,  0.00000000e+00],
       [-4.79972803e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.86921319e-05, -2.47368343e-07,  0.00000000e+00],
       ...,
       [-7.98793179e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.95180785e-05,  7.21851354e-06,  0.00000000e+00],
       [-4.57166512e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33590481e-03,  9.99352426e-05,  0.00000000e+00],
       [-5.40436492e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16595363e-04,  7.88288067e-06,  0.00000000e+00]])

In [83]:
def predict_all(X,all_theta):
    rows=X.shape[0]
    params=X.shape[1]
    num_labels=all_theta.shape[0]

    X=np.insert(X,0,np.ones(rows),axis=1)
    X=np.matrix(X)
    all_theta=np.matrix(all_theta)
    error=X*all_theta.T
    h=sigmoid(error)#m x num_labels 的矩阵，列代表每个分类器的预测结果

    h_argmax=np.argmax(h,axis=1)#取行中最大的数
    h_argmax=h_argmax+1#顶部加一行

    return h_argmax#m x 1 矩阵

In [85]:
#检验准确率
y_pred = predict_all(data['X'],all_theta)
correct=[1 if a==b else 0 for (a,b) in zip(y_pred,data['y'])]
accuray=sum(map(int,correct))/float(len(correct))
print('accuray={0}%'.format(accuray*100))

accuray=94.46%
